In [225]:
from t_graph import TGraph, TEdge, TNode
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

from measure import measure_performance, PerformanceMeasure

In [163]:
import numpy as np
import sys
import itertools

class WeightedGraphFactory():
    
    def __init__(self, num_vertices_range, num_edges_range,
                 weight_max = sys.float_info.max, weight_min = sys.float_info.min):
        self.num_vertices_range = num_vertices_range
        self.num_edges_range = num_edges_range
        self.weight_max = weight_max
        self.weight_min = weight_min
    
    def create_graph(self, num_vertices: int, num_edges: int) -> TGraph:
        
        weights = np.random.random((int(num_edges), 1))
        weights = weights * (self.weight_max - self.weight_min) + self.weight_min
        
        vertices_list = list(range(0,num_vertices))
        vertices_space = itertools.product(vertices_list, vertices_list)
        vertices_space = [e for e in vertices_space if e[0] != e[1]]
        a_vertices_space = range(0, len(vertices_space))
        
        assert len(a_vertices_space) >= num_edges, f"{num_edges} edges are too many for {num_vertices} vertices."
        
        chosen = np.random.choice(a_vertices_space, num_edges, replace=False)
        E_list = [vertices_space[i] for i in chosen]
        
        new_graph = TGraph(E_list)
        for i, e in enumerate(E_list):
            new_graph.add_edge_property(e[0], e[1], "weight", weights[i])
        
        return new_graph
    
    def __iter__(self) -> TGraph:
        for num_vertices, num_edges in zip(self.num_vertices_range,self.num_edges_range):
            yield self.create_graph(num_vertices, num_edges)

In [181]:
def create_log_grid(max_vertices, max_edges, num_points):
    vertices = np.logspace(1, np.log10(max_vertices), num_points, dtype=int)
    edges = np.logspace(1, np.log10(max_edges), num_points, dtype=int)
    


    v_mesh, e_mesh = np.meshgrid(vertices, edges)
    v_mesh = v_mesh.flatten()
    e_mesh = e_mesh.flatten()
    
    e_mesh = np.minimum(e_mesh, v_mesh * (v_mesh - 1))
    
    return v_mesh, e_mesh


In [186]:
max_vertices = 100
max_edges = 1000
num_points = 3

vertices, edges = create_log_grid(max_vertices, max_edges, num_points)

In [215]:
@measure_performance
def measure_prims_performance(graph: TGraph) -> PerformanceMeasure:
    return graph.prims_mst()

@measure_performance
def measure_kruskal_performance(graph: TGraph) -> PerformanceMeasure:
    return graph.kruskal_mst()

In [188]:
graph_factory = WeightedGraphFactory(vertices, edges, 100, 0)

In [223]:
kruskal_data = [measure_kruskal_performance(graph)[1] for graph in tqdm(graph_factory)]

9it [00:00, 232.20it/s]


In [232]:
kruskal_df = pd.DataFrame(kruskal_data, columns=["cpu_time", "elapsed_time", "memory_usage"])
kruskal_df["V"] = vertices
kruskal_df["E"] = edges